In [191]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle

In [192]:
##reading the csv file
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [193]:
data = data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [194]:
#encode the categoral value 
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data.head()


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [195]:
#one hot encoder of geography
ohe_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = ohe_geo.fit_transform(data[['Geography']]).toarray()
geo_encoder_dif = pd.DataFrame(geo_encoded,columns=ohe_geo.get_feature_names_out(['Geography']))

In [196]:
#combining the one hot encoded with original data 
data = pd.concat([data.drop('Geography',axis=1),geo_encoder_dif],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [197]:
#split the data into features & target
x = data.drop(['EstimatedSalary'],axis=1)
y=data['EstimatedSalary']

In [198]:
##split  the data into training and testing sets
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

##scale these features
scaler = StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [199]:
##save the encoders and scalers into file using the pickle package
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)
    
with open('ohe_geo.pkl','wb') as file:
    pickle.dump(ohe_geo,file)
    
with open('geo_encoder_dif.pkl','wb') as file:
    pickle.dump(geo_encoder_dif,file)

####ANN REGRESSION PROBLEM STATEMENT

In [200]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [201]:
#Build the model
model = Sequential([
    Dense(64,activation="relu",input_shape=(X_train.shape[1],)),##hidden layer1
    Dense(32,activation="relu"),##hidden layer2
    Dense(1,activation="sigmoid")##output layer
])

#compile the model
model.compile(optimizer='adam',loss="mean_absolute_error",metrics = ['mae'])
model.summary()

c:\Users\laxma\miniconda3\envs\your_env_name\lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_27 (Dense)                │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [202]:
##setup the tensorboard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

log_dir = "regressionlogs/fit/"+ datetime.datetime.now().strftime('%y%m%d-%H%M%S')
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [203]:
#set up early stopping
early_stopping_callback = EarlyStopping(monitor='val_loss',patience = 5,restore_best_weights=True)

In [204]:
##train the model

history = model.fit(
    X_train,y_train,validation_data = (X_test,y_test),epochs=100,callbacks=[tensorflow_callback,early_stopping_callback]
    
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 100430.3203 - mae: 100430.3203 - val_loss: 98725.0625 - val_mae: 98725.0625
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 100430.3125 - mae: 100430.3125 - val_loss: 98725.0625 - val_mae: 98725.0625
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 100430.2969 - mae: 100430.2969 - val_loss: 98725.0625 - val_mae: 98725.0625
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 100430.2812 - mae: 100430.2812 - val_loss: 98725.0625 - val_mae: 98725.0625
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 100430.2656 - mae: 100430.2656 - val_loss: 98725.0625 - val_mae: 98725.0625
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 100430.3125 - mae: 100430.3125 - val_loss: 98725.0625 - val_mae: 98725.0625


In [205]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [206]:
%tensorboard --logdir regressionlogs/fit --port 6006

Reusing TensorBoard on port 6006 (pid 10688), started 0:05:52 ago. (Use '!kill 10688' to kill it.)

In [207]:
#evaluate the model on test data
test_loss,test_mae=model.evaluate(X_test,y_test)
print(f'Test Mae:{test_mae}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 98725.0625 - mae: 98725.0625  
Test Mae:98725.0625


In [208]:
model.save('regression.keras')
# keras.saving.save_model(model, 'my_model.keras')